In [3]:
from data.Dataset import Retina
import torch
import torchvision
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import transforms
from torch.utils.data import SubsetRandomSampler as SRS
from torch.utils.data import DataLoader
from torchvision.models import resnet50
from torchvision.models import resnet101
import time
import copy
from torchvision.models import inception_v3
import os

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [17]:
Iv3 = inception_v3(pretrained=False)

num_classes = 6

num_in_ft1 = Iv3.AuxLogits.fc.in_features
Iv3.AuxLogits.fc = nn.Linear(num_in_ft1, num_classes)

num_in_ft2 = Iv3.fc.in_features
Iv3.fc = nn.Linear(num_in_ft2, num_classes) 

Iv3.to(device)

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

In [18]:
Iv3.load_state_dict(torch.load(os.path.join(os.getcwd(),'model','Iv3_3')))

<All keys matched successfully>

## Save to torchscript via tracing

In [21]:
# An example input would normally provide to your model's forward() method.
example = torch.randn(2,3,299,299).to(device)
Iv3 = Iv3.eval()
# Use torch.jit.trace to generate a torch.jit.ScriptModule via tracing.
traced_script_IV3 = torch.jit.trace(Iv3, example)

## Serializing script module to a file

In [23]:
traced_script_IV3.save(os.path.join(os.getcwd(),'model',"traced_Iv3_3.pt"))